In [30]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


In [75]:
NUM_CANDIDATES = 1
traces = []
for i in range(10):
    all = []
    P_REPUBLICAN = np.random.uniform(.1,.9)
    P_PARTISAN = np.random.uniform(.9,.9)
    TRUMP_PENALTY = 0
    print(
        "P_REP", P_REPUBLICAN,
        "\nP_PART", P_PARTISAN,
        "\n "
    )
    for precinct in range(int(np.random.uniform(30, 100))):
        num_voters = int(np.random.uniform(100, 1000))
        votes = pd.DataFrame(data = {"is_republican":
            np.random.binomial(1, P_REPUBLICAN, num_voters)
                             }).round()
        votes['party_votes'] = np.random.binomial(n=NUM_CANDIDATES, size=[num_voters], p=P_PARTISAN)
        votes['republican_votes'] = (
            votes['party_votes'] * votes['is_republican'] + 
            (NUM_CANDIDATES - votes['party_votes']) * (1 - votes['is_republican'])
        )
        votes['is_straight_ticket_rep'] = votes["republican_votes"] == NUM_CANDIDATES
        votes['is_straight_ticket_dem'] = votes["republican_votes"] == 0
        votes['party_president'] = np.random.binomial(n=1, size=[num_voters], p=P_PARTISAN)
        votes['republican_president'] = (
            votes['party_president'] * votes['is_republican'] + 
            (1 - votes['party_president']) * (1 - votes['is_republican'])
        )
        votes['is_straight_ticket_rep'] = votes['is_straight_ticket_rep'] * votes['republican_president']
        votes['is_straight_ticket_dem'] = votes['is_straight_ticket_dem'] * (1 - votes['republican_president'])
        totals = votes.sum()
        totals['is_straight_ticket'] = totals['is_straight_ticket_rep'] + totals['is_straight_ticket_dem']
        totals['straight_ticket_rep_pct'] = totals['is_straight_ticket_rep'] / totals['is_straight_ticket']
        totals['rep_pres_pct'] = totals['republican_president'] / num_voters
        totals['diff'] = totals['rep_pres_pct'] - totals['straight_ticket_rep_pct']
        all.append(pd.DataFrame(totals).T)
    all = pd.concat(all)
    all['c'] = P_REPUBLICAN
    traces.append(px.scatter(all, x='straight_ticket_rep_pct', y='diff', trendline="ols", color="c"))
fig = traces.pop()
for trace in traces:
    fig.add_trace(trace.data[0])
    fig.add_trace(trace.data[1])
fig.show()

P_REP 0.14436504595158084 
P_PART 0.9 
 
P_REP 0.8621308535384004 
P_PART 0.9 
 
P_REP 0.3179355073482264 
P_PART 0.9 
 
P_REP 0.37457637696220447 
P_PART 0.9 
 
P_REP 0.414678107155788 
P_PART 0.9 
 
P_REP 0.4077992837026535 
P_PART 0.9 
 
P_REP 0.8238919238095652 
P_PART 0.9 
 
P_REP 0.3970658267311421 
P_PART 0.9 
 
P_REP 0.4661781560596262 
P_PART 0.9 
 
P_REP 0.45977494665818786 
P_PART 0.9 
 


$$
p_{sr} = \frac{p_r p_p^n p_{pres}}{p_p^n (p_{pres}p_r + p_{dpres}(1-p_r))} \\ 
p_{sr} \approx p_r  
$$
if $p_{pres} \approx p_{dpres}$
$$
d = p_r p_{pres} - p_r  \\
d = p_r(p_{pres} -1) \\
$$

where $d$ is `diff`, $p_p$ is the probability of voting with your party, $n$ is the number of candidates on the ballot, $p_r$ is the probability of being a republican